In [2]:
# NLP Lecture @ Strive School - 21st July 2021
# CHATBOT with Pytorch

'''
Chatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.

Imagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.

Create a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.

The final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.

Example
A: Hello MedAssistant.
B: Hello. How can I help you?
A: I don't feel well.
B: Do you have any symptoms?
A: I have cough and nausea.
B: Do you want to book an appointment?
A: Yes, please, for tomorrow.


Info:
- Feel free to change or arrange a new dataset of intents
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)
'''

In [48]:
import numpy as np
import json

import nltk
# nltk.download()

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

In [53]:
# STEP 0: find intents patterns

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
patterns = []
for intent in intents['intents']:   
# TBD: loop through each sentence in our intents patterns, create a list of tags and define the patterns
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        wordd = tokenize(pattern)
        all_words.extend(wordd)
        patterns.append((wordd,tag))

print(all_words)
print('\n ...........................')
print(tags)
print('\n...........................')
print(patterns)

['Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Good', 'evening', 'Good', 'night', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Thank', "'s", 'a', 'lot', '!', 'Which', 'hamburgers', 'do', 'you', 'have', '?', 'What', 'kinds', 'of', 'courses', 'are', 'there', '?', 'What', 'do', 'you', 'cook', '?', 'Can', 'I', 'order', 'a', 'coke', '?', 'I', "'m", 'hungry', 'We', "'re", 'starving', 'I', "'d", 'like', 'to', 'have', 'a', 'pizza', 'Do', 'you', 'take', 'credit', 'cards', '?', 'Do', 'you', 'accept', 'Mastercard', '?', 'Can', 'I', 'pay', 'with', 'Apple', 'Pay', 'or', 'Paypal', '?', 'Can', 'I', 'pay', 'with', 'cash', '?', 'How', 'long', 'does', 'delivery', 'take', '?', 'How', 'long', 'does', 'shipping', 'take', '?', 'When', 'do', 'I', 'get', 'my', 'delivery', '?', 'Tell', 'me', 'a', 'joke', '!', 'Tell', 'me', 'something', 'funny', '!', 'Let', 'me', 'laugh', '!']

 ...........................
['greeting', 'g

In [ ]:
# STEP 1: Pre-process of the input

# lower case? stemming? stopwords?
# TBD

In [38]:
# from nltk.stem.porter import PorterStemmer
# stemmer = PorterStemmer()

In [57]:
#Ignoring punctuations 

import string
punct = string.punctuation

all_words = [stem(punc) for punc in all_words if punc not in punct]
#set removes all the repeated words
all_words = sorted(set(all_words))
tags = sorted(set(tags))
print(tags)

['delivery', 'food', 'goodbye', 'greeting', 'miscellaneous', 'payments', 'thanks']


In [64]:
# STEP 2: Define training data through a bag of words

X_train = []
y_train = []
for (pattern_sentence, tag) in patterns:

    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [68]:
# STEP 3: Configure the neural network

# define each parameter that is equal to 0 using an empirical value or a value based on your experience
# TBD

num_epochs = 1000   
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

# STEP 4: Train the model

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Define loss and optimizer: which one is the best one?
# TBD
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch[{epoch + 1}/{num_epochs}], loss:{loss.item():.4f}')
print(f'final loss: {loss.item():.4f} ')




Epoch[100/1000], loss:0.6439
Epoch[200/1000], loss:0.1789
Epoch[300/1000], loss:0.0083
Epoch[400/1000], loss:0.0038
Epoch[500/1000], loss:0.0027
Epoch[600/1000], loss:0.0014
Epoch[700/1000], loss:0.0012
Epoch[800/1000], loss:0.0003
Epoch[900/1000], loss:0.0018
Epoch[1000/1000], loss:0.0003
final loss: 0.0003 


In [70]:
# STEP 5: Save the model

# TBD: name and save the model
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}
file = 'train.pth'
torch.save(data,file)
print(f'training complete. file saved!')

training complete. file saved!


In [ ]:
# STEP 6: Test the model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# TBD: Load the intents file

# TBD: Retrieve the model and all the sizings

# TBD: build the NN

# TBD: prepare a command-line conversation (don't forget something to make the user exit the script!)